In [1]:
%matplotlib inline

import numpy as np
import collections
from collections import OrderedDict, Counter, defaultdict
import pandas as pd

import Bio
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqUtils import molecular_weight
from Bio.Alphabet import generic_dna, generic_rna, generic_protein

import seaborn as sns
import matplotlib.pyplot as plt

import glob

import subprocess
from subprocess import call

import re

import pickle

import sys                                                                         
from gffutils.iterators import DataIterator  
from gtfparse import read_gtf


Bad key text.latex.unicode in file /home/alla/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file /home/alla/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file /home/alla/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably need to get an updated matplotlibrc file from
https://githu

# Open metadata files 

In [2]:
# gencode 25 metadata 
metadata_pc_g25 = pd.read_csv('tmp_res/metadata_pc_g25.txt', sep='\t')

# gencode 35 metadata 
metadata_pc_g35 = pd.read_csv('tmp_res/metadata_pc_g35.txt', sep='\t')

# open metadata with scores and metrics
meta = pd.read_csv('tmp_res/local_and_global_df_g25ovlp_g35ovlp_g38ovlp_refseqovlp_strand_FRAMES_PhyloCSF.txt', sep='\t')

In [3]:
# genes from 2011 paper
genes_42 = 'RASL10B,FGFR1,YPEL2,ENOX2,UFSP1,WDR26,TIAL1,YPEL4,NGF,TRPV6,KCTD11,STARD10,YPEL1,R3HCC1,ZFP62,NHLRC4,EPHX3,C1QL4,TLE3,HDGF,ANKRD42,FAM217B,NFKBID,PTEN,RNF187,METTL23,TMEM8B,VANGL2,FNDC5,C1QL1,CITED1,EIF4G3,HELZ2,CYTH2,NTF3,MFSD4B,C1QL3,KCNN4,IFT46,RASD2,CITED2,C1QL2'.split(',')
genes_17 = 'GTF3A, EIF4G2, SP3, PRPS1L1, TEAD4, TEAD3, CACNG8, OAZ3, TEAD1, DDX17, VEGFA, NR1I2, HCK, WT1, BAG1, NPW, MYC'.split(', ')

# Add CDS completness and biotype='protein_coding' before selection SETs (gencode25)

In [4]:
# returns GTF with essential columns such as "feature", "seqname", "start", "end"
# alongside the names of any optional keys which appeared in the attribute column
df = read_gtf("data/gencode.v25.annotation.gtf")

# filter DataFrame to gene entries on chrY
df_genes = df[df["feature"] == "gene"]


df_CDS = df[df["feature"] == "CDS"]

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_status', 'gene_name', 'level', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_status', 'transcript_name', 'transcript_support_level', 'tag', 'havana_transcript', 'exon_number', 'exon_id', 'ont', 'protein_id', 'ccdsid']


In [5]:
annotation_genes = df_genes[['transcript_id', 'transcript_type', 'gene_name', 'tag']]
annotation_CDS = df_CDS[['transcript_id', 'transcript_type', 'gene_name', 'tag']]

annotation_CDS.columns = ['tr_id', 'transcript_type', 'gene', 'tag']

print (annotation_CDS.shape, annotation_genes.shape)

(704187, 4) (58037, 4)


In [6]:
meta1 = meta.merge(annotation_CDS, on=['tr_id', 'gene'], how='inner').drop_duplicates()

meta1.shape, meta.shape

((77513, 32), (77513, 30))

In [7]:
meta1[0:2]

,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,...,ovlp,ovlp_strandless,ovlp2,ovlp2_strandless,PhyloCSF120score,tr_id1,number_of_records,records_names,transcript_type,tag
0,ENST00000624431.1,274,316,42,274,316,14.0,FO538757.2,274,chr1:182667-182708,...,0;0;0,0;0;0,0;0;0;0,0;0;0;0,-163.9926,ENST00000624431,85,Human; Gorilla; Orangutan; Golden_snub_nosed_m...,protein_coding,"basic,appris_principal_1"
3,ENST00000624735.1,0,236,236,2,236,78.0,FO538757.1,86,chr1:200087-200236,...,0;0;0,0;0;0,0;0;0;0,0;0;0;0,-1273.9572,ENST00000624735,52,Human; Chimp; Gorilla; Orangutan; Gibbon; Pig_...,protein_coding,"basic,appris_alternative_2"


# Add predictions from Trips-Viz (ranks, proteomics support)

In [8]:
transl_with_all_codons = pd.read_csv('data/extensions_TripsViz_1perGene_AUG_NearCognates.txt', sep=',')

transl_with_all_codons.columns = ['gene_ext', 'tr_id1', 'Start_ext', 'Stop_ext', 'len_ext', 
                                 'global_rank_ext', 'type_all_codons', 'trips_viz_link', 'Start_codon_ext', 
                                 'Highframe rank', 'Highframe value',
       'Lowframe rank', 'Lowframe value', 'Stop rank', 'Stop value',
       'Start rank', 'Start value', 'Coverage rank', 'Coverage_value_ext',
       'Amino acid sequence', 'Proteomics_count_ext', 'CDS_ratio']

transl_with_all_codons2 = transl_with_all_codons[['gene_ext', 'tr_id1', 'Start_ext', 'Stop_ext', 
                                                 'global_rank_ext', 'Start_codon_ext', 
                                                 'Proteomics_count_ext', 'Coverage_value_ext', 'CDS_ratio']]

transl_with_all_codons2['Riboseq_Summary'] = transl_with_all_codons2['Start_codon_ext']+'; '+transl_with_all_codons2['Start_ext'].astype(str)+'-'+transl_with_all_codons2['Stop_ext'].astype(str)+'; Rank: '+transl_with_all_codons2['global_rank_ext'].astype(str)+'; cov: '+np.round(transl_with_all_codons2['Coverage_value_ext'],2).astype(str)


transl_with_all_codons2[0:5]

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
/home/alla/.local/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,gene_ext,tr_id1,Start_ext,Stop_ext,global_rank_ext,Start_codon_ext,Proteomics_count_ext,Coverage_value_ext,CDS_ratio,Riboseq_Summary
0,ASPH,ENST00000379454,144,189,1,CTG,0.0,100.000000,1.645069,CTG; 144-189; Rank: 1; cov: 100.0
1,HSP90AA1,ENST00000216281,168,207,2,GTG,0.0,100.000000,3.081078,GTG; 168-207; Rank: 2; cov: 100.0
2,CD24,ENST00000619133,408,441,3,CTG,0.0,83.333333,11.578498,CTG; 408-441; Rank: 3; cov: 83.33
3,HN1,ENST00000409753,242,287,4,TTG,0.0,100.000000,0.588389,TTG; 242-287; Rank: 4; cov: 100.0
4,YBX1,ENST00000321358,44,140,5,CTG,0.0,85.185185,7.709763,CTG; 44-140; Rank: 5; cov: 85.19


In [9]:
meta_cov_riboseq_pred_proteomics_pred = meta1.merge(transl_with_all_codons2, on='tr_id1', how='left')

meta_cov_riboseq_pred_proteomics_pred['Nstart_Nend'] = meta_cov_riboseq_pred_proteomics_pred['N_term_start1'].astype(str)+'-'+meta_cov_riboseq_pred_proteomics_pred['N_term_end1'].astype(str)

DF = meta_cov_riboseq_pred_proteomics_pred.copy()

print (meta_cov_riboseq_pred_proteomics_pred.shape, meta.shape)

(77513, 42) (77513, 30)


In [10]:
DF.to_csv('tmp_res/METADATA_CURRENT.txt', sep='\t', index=False)

In [11]:
DF[DF['gene'] == 'DHFR'][['tr_id', 'ovlp', 'ovlp_strandless']]

,tr_id,ovlp,ovlp_strandless
20512,ENST00000439211.6,0;0;0,237;237;237
20513,ENST00000505337.5,0;0;0,0;0;0
20514,ENST00000511032.5,0;0;0,0;0;0
20515,ENST00000504396.1,88;88;88,88;88;88


In [15]:
DF[DF['gene'] == 'DYRK1B'][['PhyloCSF120score', 'Riboseq_Summary']]

,PhyloCSF120score,Riboseq_Summary
67901,-199.8355,NaN
67902,-245.1710,NaN
67903,-245.1710,NaN
67904,-245.1710,NaN
67905,-750.3741,NaN
67906,-245.1710,NaN


In [13]:
transl_with_all_codons[transl_with_all_codons['gene_ext'] == 'DYRK1B']

,gene_ext,tr_id1,Start_ext,Stop_ext,len_ext,global_rank_ext,type_all_codons,trips_viz_link,Start_codon_ext,Highframe rank,...,Lowframe value,Stop rank,Stop value,Start rank,Start value,Coverage rank,Coverage_value_ext,Amino acid sequence,Proteomics_count_ext,CDS_ratio
9585,DYRK1B,ENST00000593685,-1,0,0,9586,extension,"<a href=""https://trips.ucc.ie/homo_sapiens/Gen...",NaN,488,...,-1,211,-1,857,-1,587,0.0,NaN,0.0,0.0


# PhyloSET 

* protein-coding biotype
* tags: complete CDS 
* no overlaps with coding exons 
* primary extension has at least 20 codons
* number of records >= 3
* PhyloCSF score > 0

In [12]:
DF = DF[DF['tag'].notna()]

* old version of SET1: we don't care about overlaps with coding exons on other strands = 60 genes vs 55 genes when we care

In [13]:
SET1_ = DF[(DF['PhyloCSF120score'] > 0) & (DF['ovlp'] == '0;0;0') & (DF['len_codons'] >= 20) &
   (DF['transcript_type'] == 'protein_coding') & (~DF['tag'].str.contains('cds_start_NF')) & 
           (~DF['tag'].str.contains('cds_end_NF')) &
    (DF['number_of_records'] >= 3) &  (DF['gene'] != 'SKI')]

In [14]:
SET1 = DF[(DF['PhyloCSF120score'] > 0) & 
   (DF['ovlp_strandless'] == '0;0;0') &
   (DF['len_codons'] >= 20) &
   (DF['transcript_type'] == 'protein_coding') &
                     (~DF['tag'].str.contains('cds_start_NF')) &
                     (~DF['tag'].str.contains('cds_end_NF')) &
    (DF['number_of_records'] >= 3) & 
    (DF['gene'] != 'SKI')]


SET1.gene.nunique()

55

In [15]:
DF[DF['tr_id'].isin(set(SET1_.tr_id.tolist()) - set(SET1.tr_id.tolist()))][['tr_id', 'gene', 'ovlp_strandless', 'global_coo_primary', 'strand', 'Riboseq_Summary', 'PhyloCSF120score']]

,tr_id,gene,ovlp_strandless,global_coo_primary,strand,Riboseq_Summary,PhyloCSF120score
3983,ENST00000635322.1,OAZ3,71;71;71,chr1:151763064-151763135,+,NaN,92.7679
3984,ENST00000627780.2,OAZ3,71;71;71,chr1:151763064-151763135,+,NaN,92.7679
3985,ENST00000321531.10,OAZ3,71;71;71,chr1:151763064-151763135,+,NaN,92.7679
56476,ENST00000624244.1,AC007906.1,89;89;89,chr16:53222653-53222742,-,ATT; 73-124; Rank: 5040; cov: 0.0,69.7054
58234,ENST00000536578.5,GLOD4,61;61;61,chr17:783266-783328,-,NaN,261.0625
71248,ENST00000374375.1,GDF5OS,77;77;77,chr20:35434081-35434158,+,AGG; 377-443; Rank: 5739; cov: 0.0,1079.0415
73164,ENST00000430524.5,RANBP1,83;83;83,chr22:20116101-20116184,+,NaN,310.2905


In [16]:
SET1_[['tr_id', 'gene', 'N_term_start1','N_term_end1', 'len_codons', 'global_coo_50_and_less', 'strand', 'global_coo_primary', 'PhyloCSF120score', 'number_of_records', 'ovlp', 'Riboseq_Summary', 
                    'Coverage_value_ext', 'Proteomics_count_ext', 'tag', 'transcript_type']].to_csv('tmp_res/SET1.txt', sep='\t', index=False)



# RiboSET

* protein-coding biotype
* tags: complete CDS 
* no overlaps with coding exons 
* primary extension has at least 20 codons
* rank is less than 500 

In [17]:
SET2_ = DF[(DF['transcript_type'] == 'protein_coding') &
                     (~DF['tag'].str.contains('cds_start_NF')) &
                     (~DF['tag'].str.contains('cds_end_NF')) & (DF['ovlp'] == '0;0;0') & 
                    (DF['global_rank_ext'] <= 500) & 
                    (DF['len_codons'] >= 20)]

print (SET2_.gene.nunique(), SET2_.tr_id.nunique())

393 395


In [18]:
SET2 = DF[(DF['transcript_type'] == 'protein_coding') &
                     (~DF['tag'].str.contains('cds_start_NF')) &
                     (~DF['tag'].str.contains('cds_end_NF')) & (DF['ovlp_strandless'] == '0;0;0') & 
                    (DF['global_rank_ext'] <= 500) & 
                    (DF['len_codons'] >= 20)]

print (SET2.gene.nunique(), SET2.tr_id.nunique())

389 391


* сheck if there is an AUG-extension 

In [19]:
SET2_[~SET2_['Riboseq_Summary'].str.contains('ATG')][['gene', 'tr_id', 'Riboseq_Summary', 'ovlp', 'ovlp_strandless']].gene.nunique()

390

In [20]:
# genes with 2 transcripts: onc chrX and chrY chromosomes 
SET2[SET2['gene'].isin(['SLC25A6', 'CD99'])][['tr_id', 'gene', 'global_coo_50_and_less']]

,tr_id,gene,global_coo_50_and_less
74759,ENST00000381401.10,SLC25A6,chrX:1392010-1392084
74779,ENST00000381192.8,CD99,chrX:2691211-2691360
77354,ENST00000381401.10_PAR_Y,SLC25A6,chrY:1392010-1392084
77374,ENST00000381192.8_PAR_Y,CD99,chrY:2691211-2691360


In [21]:
DF[DF['tr_id'].isin(set(SET2_.tr_id.tolist()) - set(SET2.tr_id.tolist()))][['tr_id', 'gene', 'ovlp_strandless', 'global_coo_primary', 'strand', 'Riboseq_Summary', 'PhyloCSF120score']]

,tr_id,gene,ovlp_strandless,global_coo_primary,strand,Riboseq_Summary,PhyloCSF120score
19446,ENST00000166345.7,TRIP13,12;12;12,chr5:892645-892998,+,GTG; 285-357; Rank: 107; cov: 73.68,-1324.6564
20512,ENST00000439211.6,DHFR,237;237;237,chr5:80654490-80654981,-,GTG; 135-495; Rank: 291; cov: 73.91,-2881.5796
31949,ENST00000297578.8,SLC25A32,109;109;0,chr8:103414938-103415099,-,TTG; 30-168; Rank: 64; cov: 92.68,-985.8394
57355,ENST00000254940.9,NIP7,121;202;121,chr16:69339431-69339829,+,ATT; 287-401; Rank: 382; cov: 60.61,-2914.6619


TRIP13 (ovlp=12nt, Rank=107, score:-1324);
DHFR (ovlp=237nt, Rank=291, score=-2881)
SLC25A32 (ovlp=109nt except RefSeq=0, Rank=64, score=-985);
NIP7 (ovlp=121nt/202nt, Rank=382, score=-2914)

In [22]:
# save it 
SET2_[['tr_id', 'gene', 'N_term_start1','N_term_end1', 'len_codons', 'global_coo_50_and_less', 'strand', 'global_coo_primary', 'PhyloCSF120score', 'number_of_records', 'ovlp', 'Riboseq_Summary', 
                    'Coverage_value_ext', 'Proteomics_count_ext', 'tag', 'transcript_type', 'CDS_ratio']].to_csv('tmp_res/SET2.txt', sep='\t', index=False)


In [23]:
DF[DF['gene'] == 'DHFR'][['tr_id', 'gene', 'Riboseq_Summary', 'ovlp', 'ovlp_strandless']]

,tr_id,gene,Riboseq_Summary,ovlp,ovlp_strandless
20512,ENST00000439211.6,DHFR,GTG; 135-495; Rank: 291; cov: 73.91,0;0;0,237;237;237
20513,ENST00000505337.5,DHFR,NaN,0;0;0,0;0;0
20514,ENST00000511032.5,DHFR,NaN,0;0;0,0;0;0
20515,ENST00000504396.1,DHFR,NaN,88;88;88,88;88;88


In [24]:
SET2_[SET2_['gene'] == 'DHFR']

,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,...,gene_ext,Start_ext,Stop_ext,global_rank_ext,Start_codon_ext,Proteomics_count_ext,Coverage_value_ext,CDS_ratio,Riboseq_Summary,Nstart_Nend
20512,ENST00000439211.6,0,494,494,2,494,164.0,DHFR,344,chr5:80654490-80654639,...,DHFR,135.0,495.0,291.0,GTG,0.0,73.913043,1.090317,GTG; 135-495; Rank: 291; cov: 73.91,2-494


In [25]:
SET2[SET2['gene'] == 'DHFR']

,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,...,gene_ext,Start_ext,Stop_ext,global_rank_ext,Start_codon_ext,Proteomics_count_ext,Coverage_value_ext,CDS_ratio,Riboseq_Summary,Nstart_Nend


# Genes that are supported by proteomics 

In [26]:
SET1_[SET1_['Proteomics_count_ext'] > 0].gene.nunique()

18

In [27]:
SET2_[SET2_['Proteomics_count_ext'] > 0].gene.nunique()

91

In [28]:
', '.join(list(set(SET1_.gene.tolist()).intersection(SET2_.gene.tolist())))

'RPTOR, HNRNPA0, CYTH2, FXR2, CCDC8, H1FX, SFPQ, MARCKS'

In [29]:
SET1_.gene.nunique(), SET2_.gene.nunique()

(60, 393)

# ATG-extensions in RiboSET

In [30]:
SET2_[SET2_['Start_codon_ext'] == 'ATG']

,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,...,gene_ext,Start_ext,Stop_ext,global_rank_ext,Start_codon_ext,Proteomics_count_ext,Coverage_value_ext,CDS_ratio,Riboseq_Summary,Nstart_Nend
17038,ENST00000467087.5,258,528,270,258,528,90.0,STIM2,378,chr4:26861069-26861218,...,STIM2,268.0,529.0,462.0,ATG,0.0,73.170732,0.139053,ATG; 268-529; Rank: 462; cov: 73.17,258-528
20891,ENST00000316788.11,251,557,306,251,557,102.0,AP3S1,407,chr5:115841888-115842037,...,AP3S1,258.0,558.0,121.0,ATG,0.0,68.421053,0.325754,ATG; 258-558; Rank: 121; cov: 68.42,251-557
39399,ENST00000418331.6,0,352,352,1,352,117.0,PTPRJ,202,chr11:47980763-47980912,...,PTPRJ,11.0,353.0,369.0,ATG,2.0,60.550459,0.407875,ATG; 11-353; Rank: 369; cov: 60.55,1-352


In [31]:
SET2_[SET2_['Start_codon_ext'] != 'ATG'].tr_id.nunique()

392

In [32]:
SET1_.tr_id.nunique(), SET1_.gene.nunique()

(85, 60)

# Add info about in-frame ATG in theoretical NTE 

In [33]:
for tr_id in list(set(SET1_.tr_id.tolist()+SET2_.tr_id.tolist())):
    #print (tr_id)
    # extract primary NTE sequence
    N_term_start = meta[meta['tr_id'] == tr_id].iloc[0].N_term_start1 
    N_term_end = meta[meta['tr_id'] == tr_id].iloc[0].N_term_end1
    NTE_seq = metadata_pc_g25[metadata_pc_g25['tr_id'] == tr_id].iloc[0].transcript_seq[N_term_start:N_term_end]
    
    codons = [NTE_seq[i:i+3] for i in range(0, len(NTE_seq), 3)]
    atgs = []
    for i, codon in enumerate(codons):
        if codon == 'ATG':
            atgs.append('ATG_'+str(i*3))
    
    if len(atgs) != 0:
        print ( metadata_pc_g25[metadata_pc_g25['tr_id'] == tr_id].iloc[0].gene, tr_id, '; '.join(atgs))

TK1 ENST00000301634.11 ATG_108
AP3S1 ENST00000316788.11 ATG_6; ATG_72
MXD4 ENST00000337190.6 ATG_144
C1GALT1 ENST00000436587.6 ATG_132
PDCD6 ENST00000264933.8 ATG_3
STIM2 ENST00000467087.5 ATG_9
SLC17A5 ENST00000355773.5 ATG_81
TMX3 ENST00000299608.6 ATG_18
CCDC127 ENST00000296824.3 ATG_15
DHX33 ENST00000225296.7 ATG_24
PTPRJ ENST00000418331.6 ATG_9; ATG_186
DUSP5 ENST00000369583.3 ATG_3
LY6K ENST00000292430.10 ATG_243
DHFR ENST00000439211.6 ATG_18; ATG_252
EID2 ENST00000390658.3 ATG_6
